# Assignment 1: Data Parsing, Cleansing and Integration
## Task 3
#### Student Name: Hongjie Xu
#### Student ID: s3880497

Date: 31/08/2021

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: 
* Pandas
* Random
* Datetime

## Introduction

In Integration Task, first of all, my plan is using `concat` method to stack two dataframe vertically. 

Therefore, the first problem is to make sure schema of both datasets be same. My solution is choosing Dataset1 we parsed and cleaned from Part2&Part3 as our global schema and resolving the schema confict in Dataset2 column by column.

Second problem is detecting and solving data-level conflicts, such as duplicates. Identifying a global key for dataset is a good to find the potential duplicates.


##  Import libraries 

In [1]:
# Code to import libraries as you need in this assessment
import pandas as pd
import re
from random import randint, randrange
from datetime import datetime

### 1. Examining and loading data
Examine "\<student\_id\>_dataset2.csv" the structure and schema. 

In [2]:
# read in two data set
# we need to make sure read datetime object in our dataframe
parse_dates=['OpenDate', 'CloseDate']
dataset1 = pd.read_csv('S3880497_dataset1_solution.csv', parse_dates=parse_dates)

dataset2 = pd.read_csv('S3880497_dataset2.csv')
# Display two dataframes together to compare
print('Dateset1:' + str(dataset1.shape))
display(dataset1.head())
print('Dataset2:' + str(dataset2.shape))
display(dataset2.head())

Dateset1:(50703, 11)


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,68187214,SQL DBA Team Lead,the city,Computer Futures,non-specified,non-specified,IT Jobs,55000.0,2012-07-24 00:00:00,2012-08-23 00:00:00,gojobsearch.co.uk
1,70099658,RGN Chiswick ****,uk,Prospect Health,non-specified,non-specified,Healthcare & Nursing Jobs,22912.0,2013-01-08 15:00:00,2013-02-07 15:00:00,gojobsearch.co.uk
2,67322981,Senior Performance Engineer,uk,ARM,non-specified,non-specified,IT Jobs,55000.0,2013-06-13 12:00:00,2013-08-12 12:00:00,gojobsearch.co.uk
3,68372925,CFD Engineer Job based in Warwickshire,rugby,Clemtech,non-specified,non-specified,Engineering Jobs,42500.0,2012-08-04 00:00:00,2012-09-03 00:00:00,gojobsearch.co.uk
4,71900386,Electromechanical Machinery Project Engineer,east sussex,Redline Group,non-specified,non-specified,Engineering Jobs,40000.0,2012-09-04 15:00:00,2012-09-18 15:00:00,gojobsearch.co.uk


Dataset2:(5000, 9)


,Job Title,Organisation,Monthly Payment,Contract Type,Category,Closing,Location,Full-Time Equivalent (FTE),Opening
0,Nursing Home Manager Thames Ditton,Liquid Personnel Ltd,2750.00,NaN,Healthcare and Nursing,2013-02-22 15:00:00,Thames Ditton,FULL TIME,2012-12-24 15:00:00
1,Deputy Manager Nursing Home RGN Shrewsbury,Liquid Personnel Ltd,2583.33,NaN,Healthcare and Nursing,2012-11-14 15:00:00,Shrewsbury,FULL TIME,2012-10-15 15:00:00
2,Deputy Manager Nursing Home (RGN) London,Liquid Personnel Ltd,3000.00,NaN,Healthcare and Nursing,2012-12-08 00:00:00,London,FULL TIME,2012-11-08 00:00:00
3,Deputy Manager (RGN) Nursing Home Bristol,Liquid Personnel Ltd,2791.67,NaN,Healthcare and Nursing,2013-01-13 15:00:00,Bristol,FULL TIME,2012-11-14 15:00:00
4,RGN or RMN Wigan,Eden Brown,2240.00,NaN,Healthcare and Nursing,2013-01-24 15:00:00,Wigan,FULL TIME,2012-11-25 15:00:00


In [3]:
print(dataset1.info())
print(dataset2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50703 entries, 0 to 50702
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Id            50703 non-null  int64         
 1   Title         50703 non-null  object        
 2   Location      50703 non-null  object        
 3   Company       50703 non-null  object        
 4   ContractType  50703 non-null  object        
 5   ContractTime  50703 non-null  object        
 6   Category      50703 non-null  object        
 7   Salary        50703 non-null  float64       
 8   OpenDate      50703 non-null  datetime64[ns]
 9   CloseDate     50703 non-null  datetime64[ns]
 10  SourceName    50703 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(7)
memory usage: 4.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column                      Non-Null Count  

From above, `Organisation`, `Contract Type`, `FTE` are identified having Null value, which should not exist.

In following process, they are needed to be addressed.

## 2. Resolving schema conflicts

### 2.  Defining an Initial Global Schema

There are many conflicts in the two schemas in Dateset1 and Dataset2, e.g., 
- Dateset1 has `Id` that stores the id of each job advertisement, however, Dataset2 does not have such information;
- `Title` in Dataset1 seems correspond to `Job Title` in Dataset2, but with different attribute names;
- Dateset1 has `Salary` that stores the annual salary of job, however, Dataset2 has monthly salary which needs to be normalized.

- `ContractType` in Dateset1 seems correspoinds to `Full-Time Equivalent (FTE)` in Dataset2, but with different attribute names and values;
- `ContractTime` in Dateset1 seems correspoinds to `Contract Type` in Dataset2, but with different attribute names;
- `OpenDate` & `CloseDate` in Dateset1 seems correspond to `Opening` & `Closing` in Dataset2, but with different attribute names;
- Dateset1 has `SourceName` that seems to store the name of job ad source, however, Dataset2 does not have such information;

### Here Dataset1 will be used as our global schema


- `Id`, 8 digits integer, no duplicates
- `Title`, String
- `Location`, String
- `Company`, String
- `ContractType`, String, only value in (`full_time`, `part_time`, `non-specified`)
- `ContractTime`, String, only value in (`permanent`, `contract`, `non-specified`)
- `Category`, String, only value in (`IT Jobs`, `Healthcare & Nursing Jobs`, `Engineering Jobs`, `Accounting & Finance Jobs`,

`Sales Jobs`, `Hospitality & Catering Jobs`, `Teaching Jobs`, `PR, Advertising & Marketing Jobs`      )
- `Salary`, float number with up to 2 demical places
- `OpenDate` & `CloseDate`, DateTime format
- `SourceName`, String

#### Conflict 1: Id in in Dataset1(Global Schema) & no Id column in Dataset2
Here give some general information about this conflict.

In global schema, we have Id , which should be a 8 digits integer

In [4]:
# Code to inspect data and identify conflicts in schema
dataset2.columns

Index(['Job Title', 'Organisation', 'Monthly Payment', 'Contract Type',
       'Category', 'Closing', 'Location', 'Full-Time Equivalent (FTE)',
       'Opening'],
      dtype='object')

In [71]:
dataset1['Id']

0        68187214
1        70099658
2        67322981
3        68372925
4        71900386
           ...   
50698    70139229
50699    68704692
50700    66928165
50701    71478475
50702    69693532
Name: Id, Length: 50703, dtype: int64

Tell me about how you fix the conflict here

In [5]:
# create a function to generater a 8 digits number

def create_N_number_id(n_digit):
    digit_start = 10**(n_digit-1)
    digit_end = (10**n_digit) - 1
    return randint(digit_start, digit_end)

create_N_number_id(8)

dataset1_id_list = dataset1.Id.to_list()

In [6]:
# Create a Id column in dataset2
dataset2['Id']=0
# insert generated 8 digits into dataset2, but note the id cannnot be duplicated or exist in dataset1

for idx, value in dataset2['Id'].items():
    used_id_list=dataset1_id_list
    temp = create_N_number_id(8)
    while temp in used_id_list:
        temp = create_N_number_id(8)
    else:
        dataset2.loc[idx, 'Id']=temp
        used_id_list.append(temp)
    
    

In [7]:
# Check id,  8 digit id
for jobid in dataset2.Id.astype(str):
    if not re.match(r'\d{8}', jobid):
        print(jobid)

No violated Id found.

#### Conflict 2: Title in Dataset1(Global Schema) & Job Title in Dataset2
Change column name for Dataset2

In [8]:
dataset2.rename(columns={'Job Title':'Title'}, inplace=True)

#### Conflict 3:  Company in Dataset1(Global Schema) & Organisation in Dataset2
Change column name for Dataset2

Note that if there is no information in this column, it should be `non-specified`.

Also we know there are 63 na value in this column from above info.

In [9]:
# Number of null value
len(dataset2[dataset2['Organisation'].isna()])

63

In [10]:
# check if nan here
dataset2[dataset2['Organisation'].isna()].head()

,Title,Organisation,Monthly Payment,Contract Type,Category,Closing,Location,Full-Time Equivalent (FTE),Opening,Id
44,Senior Engineer of Entrance Systems,NaN,4583.33,Permanent,Engineering,2013-09-08 15:00:00,Derbyshire,NaN,2013-08-09 15:00:00,29498604
45,Mechanical Design Draftsman,NaN,2291.67,Permanent,Engineering,2013-08-31 00:00:00,Greater Manchester,NaN,2013-07-02 00:00:00,57534630
46,ASPnet C Snr Developer,NaN,3666.67,Permanent,Information Technology,2012-05-26 00:00:00,Hampshire,NaN,2012-02-26 00:00:00,40042973
47,Procurement Subcontracts Manager,NaN,4791.67,Permanent,Engineering,2013-12-28 12:00:00,Hampshire,NaN,2013-09-29 12:00:00,62747061
48,Business Development Coordinator Sussex,NaN,1916.67,Permanent,Teaching,2012-07-29 12:00:00,West Sussex,NaN,2012-04-30 12:00:00,96972010


In [11]:
# replace those nan into non-specified
for idx, value in dataset2[dataset2['Organisation'].isna()]['Organisation'].items():
    dataset2.loc[idx, 'Organisation']='non-specified'

In [12]:
# check again whether any empty string or space or N/A like we found in Task1
dataset2[dataset2['Organisation'].str.fullmatch(r'[^a-zA-Z0-9]*|N/A')]['Organisation'].value_counts()

Series([], Name: Organisation, dtype: int64)

In [13]:
# Change column name to global schema
dataset2.rename(columns={'Organisation':'Company'}, inplace=True)

#### Conflict 4:  Salary in Dataset1(Global Schema) & Monthly Payment in Dataset2
Normalize montly payment in dataset2 to salary and change column name

In [14]:
# double check if nan in
dataset2[dataset2['Monthly Payment'].isna()]

,Title,Company,Monthly Payment,Contract Type,Category,Closing,Location,Full-Time Equivalent (FTE),Opening,Id


In [15]:
# Normalize monthly payment to Salary
dataset2['Monthly Payment'] = dataset2['Monthly Payment'].apply(lambda x: x*12)

In [16]:
# Rename conlumn name
dataset2.rename(columns={'Monthly Payment':'Salary'}, inplace=True)

#### Conflict 5: ContractType in Dataset1(Global Schema) & Full-Time Equivalent (FTE) in Dataset2

In dataset2, the `0.8 FTE`, `0.4 FTE`, `0.6 FTE` and `0.2 FTE` should all be replaced by `part_time` based on global schema.

Then, Change column name in dataset2.

Also, it is known that `Full-Time Equivalent (FTE)` has a lot null values from above info. 4357 In total.

In [17]:
display(dataset1['ContractType'].value_counts())
display(dataset2['Full-Time Equivalent (FTE)'].value_counts())

non-specified    37436
full_time        11752
part_time         1515
Name: ContractType, dtype: int64

FULL TIME    583
0.2 FTE       16
0.6 FTE       15
0.4 FTE       15
0.8 FTE       14
Name: Full-Time Equivalent (FTE), dtype: int64

In [18]:
# Shape of nan value
dataset2[dataset2['Full-Time Equivalent (FTE)'].isna()].shape

(4357, 10)

In [19]:
# All null values are nan, change them to required non-specified
for idx, value in dataset2[dataset2['Full-Time Equivalent (FTE)'].isna()]['Full-Time Equivalent (FTE)'].items():
    dataset2.loc[idx, 'Full-Time Equivalent (FTE)']='non-specified'

In [20]:
# change other format value to required value in global schema
dataset2['Full-Time Equivalent (FTE)'].replace({'FULL TIME':'full_time', '0.2 FTE':'part_time',
                                               '0.4 FTE':'part_time', '0.6 FTE':'part_time',
                                               '0.8 FTE':'part_time'}, inplace=True)

In [21]:
# change column name to required name in global schema
dataset2.rename(columns={"Full-Time Equivalent (FTE)":'ContractType'}, inplace=True)

#### Conflict 6: ContractTime in Dataset1(Global Schema) & ContractType in Dataset2

Change column name in dataset2, also we know there are some null values in this column. 688 intotal

In [22]:
# have a look at the possible value in two schema
display(dataset1['ContractTime'].value_counts())
display(dataset2['Contract Type'].value_counts())

permanent        30324
non-specified    14800
contract          5579
Name: ContractTime, dtype: int64

Permanent              3720
Fixed Term Contract     592
Name: Contract Type, dtype: int64

In [23]:
# shape of selected null value
dataset2[dataset2['Contract Type'].isna()].shape

(688, 10)

In [24]:
# All null value is NaN, no other format, update them into required value in global schema
for idx, value in dataset2[dataset2['Contract Type'].isna()]['Contract Type'].items():
    dataset2.loc[idx, 'Contract Type']='non-specified'

In [25]:
# Update other required value in schema
dataset2['Contract Type'].replace({'Permanent':'permanent', 'Fixed Term Contract':'contract'}, inplace=True)

In [26]:
# change column name to required name in global schema
dataset2.rename(columns={'Contract Type':'ContractTime'}, inplace=True)

In [27]:
display(dataset1['ContractTime'].value_counts())
display(dataset2['ContractTime'].value_counts())

permanent        30324
non-specified    14800
contract          5579
Name: ContractTime, dtype: int64

permanent        3720
non-specified     688
contract          592
Name: ContractTime, dtype: int64

#### Conflict 7: Category in Dataset1(Global Schema) & Category in Dataset2

No null vaue in both column from dataset2, But many inconsistencies found.

In [28]:
# have a look at category from both 
display(dataset1.Category.value_counts())

display(dataset2.Category.value_counts())

IT Jobs                             13134
Healthcare & Nursing Jobs            8183
Engineering Jobs                     7200
Accounting & Finance Jobs            6802
Sales Jobs                           4744
Hospitality & Catering Jobs          4530
Teaching Jobs                        3558
PR, Advertising & Marketing Jobs     2552
Name: Category, dtype: int64

Information Technology           1286
Engineering                      1066
Healthcare and Nursing            655
Finance and Accounting            637
Sales                             630
Hospitality and Catering          275
Teaching                          234
PR, Advertising and Marketing     217
Name: Category, dtype: int64

In [29]:
# Update wrong value to required format
dataset2['Category'].replace({'Information Technology':'IT Jobs', 
                              'Engineering':'Engineering Jobs',
                              'Healthcare and Nursing':'Healthcare & Nursing Jobs',
                              'Finance and Accounting':'Accounting & Finance Jobs',
                              'Sales':'Sales Jobs',
                              'Hospitality and Catering':'Hospitality and Catering Jobs',
                              'Teaching':'Teaching Jobs',
                              'PR, Advertising and Marketing':'PR, Advertising & Marketing Jobs'
                             }, inplace=True)

In [30]:
dataset2.Category.value_counts()

IT Jobs                             1286
Engineering Jobs                    1066
Healthcare & Nursing Jobs            655
Accounting & Finance Jobs            637
Sales Jobs                           630
Hospitality and Catering Jobs        275
Teaching Jobs                        234
PR, Advertising & Marketing Jobs     217
Name: Category, dtype: int64

#### Conflict 8: (CloseDate, OpenDate) in Dataset1(Global Schema) & (Closing, Opening) in Dataset2

No null vaue in both column from dataset2

In [31]:
# Change column name first
dataset2.rename(columns={'Closing':'CloseDate', 'Opening':'OpenDate'}, inplace=True)

In [32]:
# Check if any  invalid date in column
for idx, date in dataset2['OpenDate'].items():
    try:
        datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    except ValueError:
        print('Index',idx, "is not a valid date", date)

In [33]:
# Check if any  invalid date in column
for idx, date in dataset2['OpenDate'].items():
    try:
        datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    except ValueError:
        print('Index',idx, "is not a valid date", date)

In [34]:
# check the violated date in dataset2 again
violated_idx=[]
for date_idx in range(0,dataset2['OpenDate'].shape[0]):
    if dataset2['OpenDate'][date_idx] > dataset2['CloseDate'][date_idx]:
        violated_idx.append(date_idx)

dataset2.loc[violated_idx,:]

,Title,Company,Salary,ContractTime,Category,CloseDate,Location,ContractType,OpenDate,Id


No violated date found

In [35]:
# change to the correct datatype
dataset2['OpenDate']=pd.to_datetime(dataset2['OpenDate'], format='%Y-%m-%d %H:%M:%S')
dataset2['CloseDate']=pd.to_datetime(dataset2['CloseDate'], format='%Y-%m-%d %H:%M:%S')

In [36]:
display(dataset2.OpenDate)
display(dataset2.CloseDate)

0      2012-12-24 15:00:00
1      2012-10-15 15:00:00
2      2012-11-08 00:00:00
3      2012-11-14 15:00:00
4      2012-11-25 15:00:00
               ...        
4995   2013-09-24 15:00:00
4996   2012-05-21 12:00:00
4997   2013-11-17 15:00:00
4998   2012-01-15 12:00:00
4999   2013-04-27 15:00:00
Name: OpenDate, Length: 5000, dtype: datetime64[ns]

0      2013-02-22 15:00:00
1      2012-11-14 15:00:00
2      2012-12-08 00:00:00
3      2013-01-13 15:00:00
4      2013-01-24 15:00:00
               ...        
4995   2013-10-24 15:00:00
4996   2012-06-20 12:00:00
4997   2013-12-17 15:00:00
4998   2012-03-15 12:00:00
4999   2013-05-11 15:00:00
Name: CloseDate, Length: 5000, dtype: datetime64[ns]

#### Conflict 8: SourceName in Dataset1(Global Schema) & Non-such Column in Dataset2

In [37]:
# add a new column called SourceName, and the required value
dataset2['SourceName']='www.jobhuntlisting.com'

### Adapt datatypes across schema Dataset1 to Dataset2

In [38]:
print (dataset1.info())
print (dataset2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50703 entries, 0 to 50702
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Id            50703 non-null  int64         
 1   Title         50703 non-null  object        
 2   Location      50703 non-null  object        
 3   Company       50703 non-null  object        
 4   ContractType  50703 non-null  object        
 5   ContractTime  50703 non-null  object        
 6   Category      50703 non-null  object        
 7   Salary        50703 non-null  float64       
 8   OpenDate      50703 non-null  datetime64[ns]
 9   CloseDate     50703 non-null  datetime64[ns]
 10  SourceName    50703 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(7)
memory usage: 4.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype        

In [39]:
# Check column is consistent
list(set(dataset1.columns.to_list()) - set(dataset2.columns.to_list()))

# check datatype is consistent
for common_column in [c for c in dataset1.columns if c in dataset2.columns]: # common columns
    if dataset1[common_column].dtype == dataset2[common_column].dtype:
        print('Data type of conlumn ' + str(common_column) + ' in both dataset are same, global schema is ' + str(dataset1[common_column].dtype))

Data type of conlumn Id in both dataset are same, global schema is int64
Data type of conlumn Title in both dataset are same, global schema is object
Data type of conlumn Location in both dataset are same, global schema is object
Data type of conlumn Company in both dataset are same, global schema is object
Data type of conlumn ContractType in both dataset are same, global schema is object
Data type of conlumn ContractTime in both dataset are same, global schema is object
Data type of conlumn Category in both dataset are same, global schema is object
Data type of conlumn Salary in both dataset are same, global schema is float64
Data type of conlumn OpenDate in both dataset are same, global schema is datetime64[ns]
Data type of conlumn CloseDate in both dataset are same, global schema is datetime64[ns]
Data type of conlumn SourceName in both dataset are same, global schema is object


### 3. Merging data

In [144]:
# Code to merge two data sets
df = pd.concat([dataset1, dataset2], ignore_index=True)
print(df.shape)

(55703, 11)


### 4. Resolving data conflicts:

Categories of data conflicts are Attribute-Level

In [148]:
temp = df[['Company', 'Title']]

In [149]:
len(temp[temp.duplicated(keep=False)])

504

#### Conflict 1: (duplications)

Here lets say whether we have duplicates in the merged dataset. 

However, Considering the Id for dataset2 is created by us, it is important to ignore Id column if we are trying to find the duplicates in the dataset.

In [41]:
# Code to inspect data and identify conflicts in data
df[df.duplicated(subset=df.columns.difference(['Id']), keep=False)]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
35701,68627061,SQL Database Administrator Developer,london,Eurobase People,non-specified,permanent,IT Jobs,34000.0,2012-12-12 12:00:00,2013-01-11 12:00:00,jobserve.com
37078,68013704,SQL Database Administrator Developer,london,Eurobase People,non-specified,permanent,IT Jobs,34000.0,2012-12-12 12:00:00,2013-01-11 12:00:00,jobserve.com
49401,69535860,Internal Auditor/General Auditor,norwich,Omni Resource Management Solutions,non-specified,permanent,Accounting & Finance Jobs,40000.0,2012-07-19 12:00:00,2012-10-17 12:00:00,tntjobs.co.uk
49500,68013710,Internal Auditor/General Auditor,norwich,Omni Resource Management Solutions,non-specified,permanent,Accounting & Finance Jobs,40000.0,2012-07-19 12:00:00,2012-10-17 12:00:00,tntjobs.co.uk


we found four records regarding two duplicates,

-Index `35701` is same as Id`37078`, regarding all other columns.

-Id `49401` is same as Id`49500`, regarding all other columns.

Their index shows they are all from dataset1, but not dataset2.

From index of thir We decided to drop `35701` & `49401` here.

In [42]:
# drop duplicates,  so that we can make sure 
df.drop(df.index[[35701, 49401]], inplace=True)


# Check again
df[df.duplicated(subset=df.columns.difference(['Id']), keep=False)]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName


In [110]:
df[df.duplicated(subset=df.columns.difference(['Id']), keep=False)]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
35701,68627061,SQL Database Administrator Developer,london,Eurobase People,non-specified,permanent,IT Jobs,34000.0,2012-12-12 12:00:00,2013-01-11 12:00:00,jobserve.com
37078,68013704,SQL Database Administrator Developer,london,Eurobase People,non-specified,permanent,IT Jobs,34000.0,2012-12-12 12:00:00,2013-01-11 12:00:00,jobserve.com
49401,69535860,Internal Auditor/General Auditor,norwich,Omni Resource Management Solutions,non-specified,permanent,Accounting & Finance Jobs,40000.0,2012-07-19 12:00:00,2012-10-17 12:00:00,tntjobs.co.uk
49500,68013710,Internal Auditor/General Auditor,norwich,Omni Resource Management Solutions,non-specified,permanent,Accounting & Finance Jobs,40000.0,2012-07-19 12:00:00,2012-10-17 12:00:00,tntjobs.co.uk


In [96]:
dataset1[(dataset1['Id']==72450080)]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
612,72450080,Sales Coordinator Colliers Wood,south east london,Orange Recruitment,non-specified,permanent,Sales Jobs,25000.0,2013-09-09 12:00:00,2013-12-08 12:00:00,totaljobs.com


In [97]:
dataset2[dataset2['Company']=='Orange Recruitment']

,Title,Company,Salary,ContractTime,Category,CloseDate,Location,ContractType,OpenDate,Id,SourceName
276,"Credit Controller Farsley, Leeds",Orange Recruitment,20499.96,permanent,Accounting & Finance Jobs,2012-06-12 15:00:00,Leeds,non-specified,2012-03-14 15:00:00,57507085,www.jobhuntlisting.com
2607,English Teacher West Midlands,Orange Recruitment,32400.00,contract,Teaching Jobs,2012-07-17 00:00:00,UK,non-specified,2012-06-17 00:00:00,31843724,www.jobhuntlisting.com
2619,Sales Account Manager Eltham,Orange Recruitment,35000.04,permanent,Sales Jobs,2014-01-09 00:00:00,East Sheen,non-specified,2013-11-10 00:00:00,27681808,www.jobhuntlisting.com
2851,Sales Coordinator Colliers Wood,Orange Recruitment,24999.96,permanent,Sales Jobs,2013-12-08 12:00:00,South East London,non-specified,2013-09-09 12:00:00,40398428,www.jobhuntlisting.com
2918,Portfolio Executive (Collections) Newcastle u...,Orange Recruitment,20000.04,permanent,Accounting & Finance Jobs,2012-11-30 12:00:00,Newcastle Upon Tyne,non-specified,2012-09-01 12:00:00,33269594,www.jobhuntlisting.com
3574,Electronic Security Service Engineer Caerphilly,Orange Recruitment,21500.04,permanent,Engineering Jobs,2014-01-19 12:00:00,Caerphilly,non-specified,2013-10-21 12:00:00,93910300,www.jobhuntlisting.com
3575,Data Communications Engineer Caerphilly,Orange Recruitment,27500.04,permanent,Engineering Jobs,2013-01-10 15:00:00,Caerphilly,non-specified,2012-10-12 15:00:00,43620687,www.jobhuntlisting.com


#### Finding global key for the data
Find a proper global key for the integrated data and give justification here.

In [43]:
# Code to identify global key
len(df.Id.unique()) == len(df.Id)

True

In [44]:
df.Id.value_counts()

27136002    1
69560070    1
71306987    1
69089004    1
69169982    1
           ..
55410042    1
69747402    1
69162367    1
68258799    1
64527060    1
Name: Id, Length: 55701, dtype: int64

Each number in Id is unique, therefore, we can choose `Id` column as the indentified **global key** in this dataset.

### 5. Saving the integrated and reshaped data
The last part of the integration process is to export our output data to csv format, named as:
- '\<student\_id\>_dataset_integrated.csv'

In [45]:
# code to save output data
df.to_csv('S3880497_dataset_integrated.csv', index=False)

## Summary of the Assessment Task

In Task 3, the main challenge is identifying the global schema. Once global schema is identified, mapping other format to the identified one is not much hard. 

- Must be very carefull when mapping the schema, because the integration may fail because of some minor errors
- Definiation of duplicates is based on the task requirement, here, in this assignment, I believe an exactly same job cannot have two Id in one single website source. 
- Datatype is another important element to care about.

Overall, integration is not easier than cleanining because it requires to think about the consistency between two different datasets. One minor error may fail the integration stage. 
